# Historical data Processing

To process large amounts of data that we have stored, sometimes our local machine is not enough. So, the below notebook was uploaded on a Dataproc cluster on Google Cloud for processing.

The Dataproc cluster consisted of 1 master node and 5 worker nodes (E2 -4CPU), and the processing was done much faster.

The files were uploaded on a connected GCS bucket and were accessed from there.
We could also query the data directly from Firestore (upload the access key for the DB), but Firestore would charge all the reads, so this option was not preferred, since the data was already stored in my local disk and not only in the Firestore DB.


In [1]:
import os, pandas as pd
import numpy as np
import datetime as dt
import time

from pyspark.context import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
#create spark context
conf = SparkConf().setAppName('final_project')#.setMaster('yarn')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

## Load Data

Loaded the data using below schemata to cast the data types. Loaded both in DF and RDD format.

In [5]:
#Directory for the GCS bucket
gcs = "gs://final-project-372202/"

In [6]:
user_dir= gcs+'data/users.json'
user_schema=StructType([
    StructField('sid', LongType(), True),
    StructField('profile_id', LongType(), True),
    StructField('profile_name', StringType(), True),
    StructField('firstname_lastname', StringType(), True),
    StructField('description', StringType(), True),
    StructField('following', IntegerType(), True),
    StructField('followers', IntegerType(), True),
    StructField('n_posts', IntegerType(), True),
    StructField('url', StringType(), True),
    StructField('cts', TimestampType(), True),
    StructField('is_business_account', StringType(), True)
])

In [7]:
loc_dir= gcs+'data/locations.json'
loc_schema=StructType([
    StructField('sid', LongType(), True),
    StructField('id', LongType(), True),
    StructField('name', StringType(), True),
    StructField('street', StringType(), True),
    StructField('zip', StringType(), True),
    StructField('city', StringType(), True),
    StructField('region', StringType(), True),
    StructField('cd', StringType(), True),
    StructField('phone', StringType(), True),
    StructField('aj_exact_city_match', StringType(), True),
    StructField('aj_exact_country_match', StringType(), True),
    StructField('blurb', StringType(), True),
    StructField('dir_city_id', StringType(), True),
    StructField('dir_city_name', StringType(), True),
    StructField('dir_city_slug', StringType(), True),
    StructField('dir_country_id', StringType(), True),
    StructField('dir_country_name', StringType(), True),
    StructField('lat', DoubleType(), True), 
    StructField('lng', DoubleType(), True),
    StructField('primary_alias_on_fb', StringType(), True),
    StructField('slug', StringType(), True),
    StructField('website', StringType(), True),
    StructField('cts', TimestampType(), True)
])

In [8]:
post_dir= gcs+'data/posts.json'
post_schema = StructType([
    StructField('sid', IntegerType(), False),
    StructField('sid_profile', IntegerType(), False),
    StructField('post_id', StringType(), False),
    StructField('profile_id', IntegerType(), False),
    StructField('location_id', IntegerType(), True),
    StructField('cts', TimestampType(), False),
    StructField('post_type', IntegerType(), True),
    StructField('description', StringType(), True),
    StructField('numbr_likes', IntegerType(), True),
    StructField('number_comments', IntegerType(), True)
])

In [9]:
users_df=spark.read.json(user_dir,schema=user_schema)
loc_df=spark.read.json(loc_dir,schema=loc_schema)
posts_df=spark.read.json(post_dir,schema=post_schema)

In [10]:
users_rdd=users_df.rdd
users_rdd.getNumPartitions()

9

In [11]:
loc_rdd=loc_df.rdd
loc_rdd.getNumPartitions()

4

In [12]:
posts_rdd=posts_df.rdd
posts_rdd.getNumPartitions()

131

## Data Processing

### RDD MapReduce

Let's find which locations were the most popular each year. We will consider the number of likes, comments, and posts in each location to achieve this.

First, we need to filter the posts that have the location saved and map each post document to < (year, location_id), (number_likes, number_comments, 1) > tuples.

In [13]:
posts_with_loc=posts_rdd.filter(lambda doc: doc.location_id!=None).filter(lambda doc: doc.cts !=None)

In [14]:
posts_with_loc.persist()
posts_with_loc.count()

20426081

In [16]:
def map1(doc):
    year=doc.cts.year
    loc_id=doc.location_id
    likes=doc.numbr_likes
    comments=doc.number_comments
    return (year,loc_id),[likes,comments,1]
    

In [17]:
from operator import add

def add_lists(l1,l2):
    l1=[0 if i is None else i for i in l1]
    l2=[0 if i is None else i for i in l2]
    
    return list(map(add,l1,l2))

In [18]:
results=posts_with_loc.map(map1)\
                            .reduceByKey(add_lists)\
                            .map(lambda t:( t[0][1],[t[0][0]]+t[1]))  #unpack with loc_id as key for joining

In [20]:
posts_with_loc.unpersist()
results.take(5)

[(1029033463, [2017, 408, 27, 3]),
 (288813140, [2017, 279, 23, 2]),
 (265975287, [2017, 80, 0, 1]),
 (4222676, [2019, 320, 10, 26]),
 (333297550, [2018, 303, 24, 2])]

In [21]:
#Keep only relevant fields
summarized_locs=loc_rdd.map(lambda doc: (doc.id,[ doc.city,doc.cd]))

In [22]:
summarized_locs.take(5)

[(110296492939207, [None, None]),
 (274391278, ['Nová Vieska', 'SK']),
 (148885595789195, ['Kathmandu, Nepal', 'NP']),
 (263258277, ['Paris, France', 'FR']),
 (406147529857708, ['Burbank, California', 'US'])]

In [23]:
joined=results.join(summarized_locs)

In [24]:
joined.take(5)

[(674262360, ([2019, 10263, 414, 29], ['Hermosillo, Sonora', 'MX'])),
 (674262360, ([2018, 275, 18, 2], ['Hermosillo, Sonora', 'MX'])),
 (674262360, ([2016, 149, 18, 1], ['Hermosillo, Sonora', 'MX'])),
 (674262360, ([2017, 269, 37, 2], ['Hermosillo, Sonora', 'MX'])),
 (5123790, ([2016, 763, 26, 4], ['Covington, Georgia', 'US']))]

Lets reduce one more time since many locations could be in the same city. (MR done 2 times so that we could join with fewer datapoints above)

In [25]:
joined2=joined.map(lambda t: (tuple(t[1][1]+[t[1][0][0]]),t[1][0][1:]))\ #key(city,country,year),values(likes,com,posts)
                .reduceByKey(add_lists)\
                .map(lambda t: list(t[0])+t[1])\
                #.sortBy(lambda t: t[-1] if t[-1]!=None else 0,ascending=False)

In [26]:
joined2.take(5)

[['Columbus, Ohio', 'US', 2019, 2001624, 65949, 9606],
 ['Maghar, Israel', 'IL', 2019, 2621, 282, 26],
 ['Addis Ababa, Ethiopia', 'ET', 2018, 214948, 1840, 107],
 ['Leeds', 'GB', 2017, 605531, 12885, 1663],
 ['Varadero, Cuba', 'CU', 2017, 40157, 961, 69]]

In [27]:
joined2.count()

362935

In [28]:
res_df=joined2.toDF()

We significantly reduced the datapoints from 42M to 362K rows. Let's save the results to disk for further analysis.

In [32]:
res_df.coalesce(1).write.csv(gcs+'/data/MR_results.csv',header=True)

### Spark DataFrames

A similar procedure can be followed by using the Spark Dataframe API that utilizes the Catalyst optimizer. This way was way more intuitive and easy to process. This time lets also calculate the average number of likes per city and groupby months.

In [33]:
posts_df=posts_df.dropna(subset=['location_id'])

In [35]:
posts_df.printSchema()

root
 |-- sid: integer (nullable = true)
 |-- sid_profile: integer (nullable = true)
 |-- post_id: string (nullable = true)
 |-- profile_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- cts: timestamp (nullable = true)
 |-- post_type: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- numbr_likes: integer (nullable = true)
 |-- number_comments: integer (nullable = true)



In [38]:
#Summarize the locations to keep only the attributes we will use
summarized_post_df=posts_df.select(['sid','location_id',year('cts'),month('cts'),'numbr_likes','number_comments'])
summarized_locs_df=loc_df.select(['id','city','cd'])

In [41]:
summarized_post_df.show(5)
summarized_locs_df.show(5)

+--------+-----------+---------+----------+-----------+---------------+
|     sid|location_id|year(cts)|month(cts)|numbr_likes|number_comments|
+--------+-----------+---------+----------+-----------+---------------+
|28370964|  282618748|     2018|        12|        138|             15|
|28370957|  282618748|     2018|         8|        389|             10|
|28370960|  282618748|     2018|         8|        198|             23|
|28370961|  282618748|     2015|         5|        127|              8|
|28370962|  282618748|     2017|         1|        154|              6|
+--------+-----------+---------+----------+-----------+---------------+
only showing top 5 rows

+---------------+-------------------+----+
|             id|               city|  cd|
+---------------+-------------------+----+
|110296492939207|               null|null|
|      274391278|        Nová Vieska|  SK|
|148885595789195|   Kathmandu, Nepal|  NP|
|      263258277|      Paris, France|  FR|
|406147529857708|Burbank, 

In [42]:
#Join the two tables
joined_df=summarized_post_df.join(summarized_locs_df,
             summarized_post_df.location_id== summarized_locs_df.id,
            how='inner')

In [43]:
joined_df.select(countDistinct('location_id')).show()

+---------------------------+
|count(DISTINCT location_id)|
+---------------------------+
|                     618138|
+---------------------------+



In [45]:
joined_df.show(5)

+--------+-----------+---------+----------+-----------+---------------+---+--------------------+---+
|     sid|location_id|year(cts)|month(cts)|numbr_likes|number_comments| id|                city| cd|
+--------+-----------+---------+----------+-----------+---------------+---+--------------------+---+
| 7083806|         19|     2019|         3|         10|              0| 19|San Francisco, Ca...| US|
| 7083807|         19|     2019|         3|         19|              0| 19|San Francisco, Ca...| US|
| 4973895|         19|     2019|         3|        102|              4| 19|San Francisco, Ca...| US|
| 6304808|         19|     2018|        12|        105|              3| 19|San Francisco, Ca...| US|
|41835858|         19|     2019|         6|         14|              2| 19|San Francisco, Ca...| US|
+--------+-----------+---------+----------+-----------+---------------+---+--------------------+---+
only showing top 5 rows



In [57]:
#Calculate the aggregations

agg_stats=joined_df.select(['year(cts)','month(cts)','city','cd','numbr_likes','number_comments'])\
                .groupBy(['year(cts)','month(cts)','city','cd'])\
                .agg(count("city").alias("total_posts_per_city"), \
                        sum("numbr_likes").alias("total_likes"),\
                        round(avg("numbr_likes"),2).alias("avg_likes"))\
                .filter(col('cd').like('__'))\
                .sort(desc('total_posts_per_city'))

In [58]:
agg_stats.show()

+---------+----------+--------------------+---+--------------------+-----------+---------+
|year(cts)|month(cts)|                city| cd|total_posts_per_city|total_likes|avg_likes|
+---------+----------+--------------------+---+--------------------+-----------+---------+
|     2019|         5|      Moscow, Russia| RU|              182184|   64868094|    356.4|
|     2019|         5|London, United Ki...| GB|              175010|   49667834|   283.95|
|     2019|         3|  New York, New York| US|              143630|   51300433|   357.39|
|     2019|         4|London, United Ki...| GB|              111596|   31503513|   282.48|
|     2019|         4|      Moscow, Russia| RU|               95125|   43006660|   452.62|
|     2019|         6|      Moscow, Russia| RU|               88128|   25869840|   293.76|
|     2019|         4|  New York, New York| US|               84364|   22231446|   263.66|
|     2019|         2|  New York, New York| US|               76779|   30469687|   397.11|

Looks like that May 2019 in Moscow had the most posts, just above London's posts at the same month. Also, we can only see year=2019 in the top20 rows, which shows that instagram's usage was becoming higher each year. (2019 was the latest year in the dataset so it would contain more posts)

More insights and visualizations can be found on a report I prepared on Google Looker at:  
https://datastudio.google.com/reporting/e647d5ac-e2e2-437f-ac48-cb63d82fe382/page/p_zbw36zck2c

In [59]:
agg_stats.coalesce(1).write.csv(gcs+'/data/agg_results.csv',header=True)

In [60]:
agg_stats.count()

1377836